In [1]:
import numpy as np
from paths_tools import LatticePaths
from pvar_tools import *
import time

In [2]:
x = np.random.rand(10, 2)
y = np.random.rand(9, 2)

In [3]:
LP = LatticePaths(x, y, brute_force=True)

In [4]:
LP.total_paths

24310

In [5]:
warping_path = LP.allPaths[876]

In [6]:
x_reparam, y_reparam = LP.align(warping_path)

In [7]:
t=time.time()
print(p_variation_distance(x_reparam, y_reparam, p=2, depth=2, norm='l1'))
print(time.time()-t)

1 2
2 3
3 4
4 5
5 6
6 6
6 7
7 8
8 9
9 10
10 11
11 11
11 11
11 11
11 11
11 11
11 11
(5.939921536078252, [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 17])
0.012964487075805664


In [8]:
t=time.time()
print(p_variation_distance_alexey_optim(x_reparam, y_reparam, p=2, depth=2, norm='l1'))
print(time.time()-t)

(5.939921536078252, [0, 1, 2, 3, 6, 7, 8, 10, 12, 14, 16, 17])
0.0059854984283447266
